# Downloading the data

This notebook is used to download the ERA5 data from weatherbench2.


In [17]:
import xarray as xr
import os
import fsspec
import numpy as np
import pandas as pd
import jinja2

In [2]:
#!gcloud auth login
#!gsutil -m cp -r "gs://weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_conservative.zarr" ./ERA5_data

/bin/bash: line 1: gcloud: command not found


In [6]:
# testing zarr:
ds = xr.Dataset(
    {"foo": (("x", "y"), np.random.rand(4, 5))},
    coords={
        "x": [10, 20, 30, 40],
        "y": pd.date_range("2000-01-01", periods=5),
        "z": ("x", list("abcd")),
    },
)

ds.to_zarr("../test/test.zarr")

In [7]:
ds_zarr = xr.open_zarr("../test/test.zarr")
ds_zarr

ImportError: Dask diagnostics requirements are not installed.

Please either conda or pip install as follows:

  conda install dask                     # either conda install
  python -m pip install "dask[diagnostics]" --upgrade  # or python -m pip install

<xarray.Dataset> Size: 248B
Dimensions:  (x: 4, y: 5)
Coordinates:
  * x        (x) int64 32B 10 20 30 40
  * y        (y) datetime64[ns] 40B 2000-01-01 2000-01-02 ... 2000-01-05
    z        (x) <U1 16B dask.array<chunksize=(4,), meta=np.ndarray>
Data variables:
    foo      (x, y) float64 160B dask.array<chunksize=(4, 5), meta=np.ndarray>

In [12]:

# Define the relative path to the file
relative_path = "ERA5_data/1959-2022-6h-64x32_equiangular_conservative.zarr"

# Get the absolute path by joining the current directory with the relative path
absolute_path = os.path.join(os.path.dirname(os.getcwd()), relative_path)
print(absolute_path)


/home/jupyter-lukas/Masters-Thesis/ERA5_data/1959-2022-6h-64x32_equiangular_conservative.zarr


In [16]:

# Define the path to the Zarr file
file_path = "Masters-Thesis/ERA5_data/1959-2022-6h-64x32_equiangular_conservative.zarr"

# Open the Zarr file using xarray
obs_data = xr.open_zarr(absolute_path)
obs_data


ImportError: Dask diagnostics requirements are not installed.

Please either conda or pip install as follows:

  conda install dask                     # either conda install
  python -m pip install "dask[diagnostics]" --upgrade  # or python -m pip install

<xarray.Dataset> Size: 81GB
Dimensions:                                           (time: 92044,
                                                       longitude: 64,
                                                       latitude: 32, level: 13)
Coordinates:
  * latitude                                          (latitude) float64 256B ...
  * level                                             (level) int64 104B 50 ....
  * longitude                                         (longitude) float64 512B ...
  * time                                              (time) datetime64[ns] 736kB ...
Data variables: (12/38)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 754MB dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind                           (time, longitude, latitude) float32 754MB dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    10m_wind_speed                                    (time, longitude, latitude) float

In [ ]:
# prune dataset of unnecessary variables.
for x in obs_data:
    if x not in ['longitude', 'latitude', 'time', 't2m', 'msl', 'u10', 'v10']:
        obs_data = obs_data.drop_vars(x)

obs_data

In [ ]:
#rename obs data:
obs_data = obs_data.rename({'longitude': 'lon'})
obs_data = obs_data.rename({'latitude': 'lat'})

# add datetime coord:
copied_coord = obs_data['time'].copy()
# Rename the copied coordinate to the new name
obs_data = obs_data.assign_coords(datetime=copied_coord)

# add batch dim:
obs_data = obs_data.expand_dims('batch')